In [110]:
import numpy as np
import math

In [111]:
from ase.io import read
atoms=read('data.vasp',format='vasp')

#all atom is in central region by definition
#here we assume number of ghost shell is 1

#construct the list of vector connecting between each atoms

masses=atoms.get_masses()
natom=len(atoms.positions)

dist=np.zeros((natom,natom,3))
for i in range(natom):
    for j in range(natom):
        rmin=np.zeros(3)
        rij=atoms.get_distance(i,j,vector=True)
        rij_mic=atoms.get_distance(i,j,mic=True,vector=True)
        if(np.linalg.norm(rij_mic)<np.linalg.norm(rij)):
            rmin=rij_mic
        else:
            rmin=rij
        dist[i,j]=rmin

In [112]:
with open('FORCE_CONSTANTS_2ND','r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            fc_all[i,j]=fc/math.sqrt(masses[i]*masses[j])
            #fc_all[i,j]=fc
            start=start+nlineblock



In [113]:
fc_all[0,1]

array([[ 0.03913539, -0.        , -0.        ],
       [ 0.        , -0.00978188, -0.        ],
       [ 0.        , -0.        , -0.00978188]])

In [114]:
atoms.get_masses()

array([28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085, 28.085])

In [115]:
dist

array([[[ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 , -2.725 ],
        [-2.725 ,  0.    , -2.725 ],
        [-2.725 , -2.725 ,  0.    ],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625]],

       [[ 0.    ,  2.725 ,  2.725 ],
        [ 0.    ,  0.    ,  0.    ],
        [-2.725 ,  2.725 ,  0.    ],
        [-2.725 ,  0.    ,  2.725 ],
        [ 1.3625, -1.3625, -1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [-1.3625, -1.3625,  1.3625],
        [-1.3625,  1.3625, -1.3625]],

       [[ 2.725 ,  0.    ,  2.725 ],
        [ 2.725 , -2.725 ,  0.    ],
        [ 0.    ,  0.    ,  0.    ],
        [ 0.    , -2.725 ,  2.725 ],
        [-1.3625,  1.3625, -1.3625],
        [-1.3625, -1.3625,  1.3625],
        [ 1.3625,  1.3625,  1.3625],
        [ 1.3625, -1.3625, -1.3625]],

       [[ 2.725 ,  2.725 ,  0.    ],
        [ 2.725 ,  0.    , -2.725 ],
        [ 0.    ,  2.725 , -2.72

In [116]:
#slice for x-component
dist[:,:,0]

array([[ 0.    ,  0.    , -2.725 , -2.725 ,  1.3625,  1.3625, -1.3625,
        -1.3625],
       [ 0.    ,  0.    , -2.725 , -2.725 ,  1.3625,  1.3625, -1.3625,
        -1.3625],
       [ 2.725 ,  2.725 ,  0.    ,  0.    , -1.3625, -1.3625,  1.3625,
         1.3625],
       [ 2.725 ,  2.725 ,  0.    ,  0.    , -1.3625, -1.3625,  1.3625,
         1.3625],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  2.725 ,
         2.725 ],
       [-1.3625, -1.3625,  1.3625,  1.3625,  0.    ,  0.    ,  2.725 ,
         2.725 ],
       [ 1.3625,  1.3625, -1.3625, -1.3625, -2.725 , -2.725 ,  0.    ,
         0.    ],
       [ 1.3625,  1.3625, -1.3625, -1.3625, -2.725 , -2.725 ,  0.    ,
         0.    ]])

In [117]:
Rx=dist[:,:,0]
Ry=dist[:,:,1]
Rz=dist[:,:,2]
Rx.shape, fc_all.shape

((8, 8), (8, 8, 3, 3))

In [118]:
Vx=np.zeros((natom,natom,3,3))
Vy=np.zeros((natom,natom,3,3))
Vz=np.zeros((natom,natom,3,3))

for i in range(natom):
    for j in range(natom):
        Vijx=Rx[i,j]*fc_all[i,j]
        Vx[i,j]=Vijx

        Vijy=Ry[i,j]*fc_all[i,j]
        Vy[i,j]=Vijy

        Vijz=Rz[i,j]*fc_all[i,j]
        Vz[i,j]=Vijz



In [119]:
#compare with GULP output
#1st atom x-direction component
Vx[0,:,0,:]

array([[ 0.        , -0.        , -0.        ],
       [ 0.        , -0.        , -0.        ],
       [ 0.02665562, -0.        ,  0.        ],
       [ 0.02665562,  0.        , -0.        ],
       [-0.21791282, -0.13620392, -0.13620392],
       [-0.21791282,  0.13620392,  0.13620392],
       [ 0.21791282, -0.13620392,  0.13620392],
       [ 0.21791282,  0.13620392, -0.13620392]])

GULP side
```
 ixyz:           1
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  0.000000000000000E+000  0.000000000000000E+000  0.000000000000000E+000
  2.665087135698760E-002 -8.616184384773376E-017  7.000649812628367E-017
  2.665087135698760E-002  7.000649812628367E-017 -8.616184384773376E-017
 -0.217874033757599      -0.136179679902043      -0.136179679902043     
 -0.217874033757600       0.136179679902044       0.136179679902044     
  0.217874033757599      -0.136179679902043       0.136179679902043     
  0.217874033757599       0.136179679902043      -0.136179679902043     
```

In [120]:
#1st atom y-direction component
Vx[0,:,1,:]

array([[-0.        ,  0.        , -0.        ],
       [ 0.        , -0.        , -0.        ],
       [ 0.        , -0.10664395,  0.        ],
       [ 0.        ,  0.02665562, -0.        ],
       [-0.13620392, -0.21791282, -0.13620392],
       [ 0.13620392, -0.21791282, -0.13620392],
       [-0.13620392,  0.21791282, -0.13620392],
       [ 0.13620392,  0.21791282, -0.13620392]])

GULP
```
0.00E+00	0.00E+00	0.00E+00
0.00E+00	0.00E+00	0.00E+00
8.62E-17	-0.106624965	8.62E-17
7.00E-17	2.67E-02	-8.62E-17
-0.13617968	-0.217874034	-0.13617968
0.13617968	-0.217874034	-0.13617968
-0.13617968	0.217874034	-0.13617968
0.13617968	0.217874034	-0.13617968
```

In [121]:
Vx[7,:,2,:]

array([[ 0.13620392,  0.13620392, -0.21791282],
       [-0.13620392,  0.13620392, -0.21791282],
       [-0.13620392,  0.13620392,  0.21791282],
       [ 0.13620392,  0.13620392,  0.21791282],
       [-0.        ,  0.        , -0.10664395],
       [ 0.        ,  0.        ,  0.02665562],
       [-0.        , -0.        , -0.        ],
       [ 0.        , -0.        ,  0.        ]])

GULP
```
0.13617968	0.13617968	-0.217874034
-0.13617968	0.13617968	-0.217874034
-0.13617968	0.13617968	0.217874034
0.13617968	0.13617968	0.217874034
0.00E+00	-5.39E-18	0.106624965
-3.23E-17	-1.62E-17	-2.67E-02
0.00E+00	0.00E+00	0.00E+00
0.00E+00	0.00E+00	0.00E+00
```

In [122]:
#y-component

Vy[0,:,0,:]

array([[ 0.        , -0.        , -0.        ],
       [-0.10664395,  0.        ,  0.        ],
       [-0.        ,  0.        , -0.        ],
       [ 0.02665562,  0.        , -0.        ],
       [-0.21791282, -0.13620392, -0.13620392],
       [ 0.21791282, -0.13620392, -0.13620392],
       [-0.21791282,  0.13620392, -0.13620392],
       [ 0.21791282,  0.13620392, -0.13620392]])

GULP
```
0.00E+00	0.00E+00	0.00E+00
-0.106624965	8.62E-17	8.62E-17
0.00E+00	0.00E+00	0.00E+00
2.67E-02	7.00E-17	-8.62E-17
-0.217874034	-0.13617968	-0.13617968
0.217874034	-0.13617968	-0.13617968
-0.217874034	0.13617968	-0.13617968
0.217874034	0.13617968	-0.13617968
```

In [123]:
#z component
Vz[0,:,0,:]

array([[ 0.        , -0.        , -0.        ],
       [-0.10664395,  0.        ,  0.        ],
       [ 0.02665562, -0.        ,  0.        ],
       [-0.        , -0.        ,  0.        ],
       [-0.21791282, -0.13620392, -0.13620392],
       [ 0.21791282, -0.13620392, -0.13620392],
       [ 0.21791282, -0.13620392,  0.13620392],
       [-0.21791282, -0.13620392,  0.13620392]])

GULP
```
0.00E+00	0.00E+00	0.00E+00
-0.106624965	8.62E-17	8.62E-17
2.67E-02	-8.62E-17	7.00E-17
0.00E+00	0.00E+00	0.00E+00
-0.217874034	-0.13617968	-0.13617968
0.217874034	-0.13617968	-0.13617968
0.217874034	-0.13617968	0.13617968
-0.217874034	-0.13617968	0.13617968
```